# Overhead Signs Work Order Creation
--------------------------------------------------
The purpose of this script is to create work orders of overhead street name signs. The information for work orders are created from a feature layer of operational maintenance areas intersecting signalized intersections.

Disclaimer: 

## Imports

In [2]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer,GeoAccessor, GeoSeriesAccessor
from arcgis.features.manage_data import overlay_layers
from pathlib import Path
import datetime
import pandas as pd
import openpyxl
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import io
import os
import sys
from google.cloud import automl_v1beta1
from google.cloud.automl_v1beta1.proto import service_pb2
from google.cloud import vision
from google.cloud.vision import types

/anaconda3/envs/py382/lib/python3.8/site-packages/arcgis/features/_data/geodataset/utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


## Constants
Set constants of the notebook.

In [3]:
URL = r"https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/{}/FeatureServer/0"
AUTHOR = "Susanne Gov"
FILE_PATH = str(Path.cwd())
YEAR = 2020
TABLE_NAME = FILE_PATH + r"/OverheadSigns_FY{}.xlsx".format(str(YEAR))
DIRECTIONS = list("NSEW")

## Part 1: Setup Table

<b>This step can be skipped if the Excel file is already in the folder. </b>

Set variables for analysis layer title and columns for table.

In [3]:
overhead = "Signs Markings Maintenance"
col = ['COA_INTERSECTION_ID','FY_OVERHEAD_SIGN_MAINT','LOCATION_NAME','LATITUDE','LONGITUDE']

Acccess COA account

In [9]:
gis = GIS("https://austin.maps.arcgis.com/", client_id='CrnxPfTcm7Y7ZGl7')

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://austin.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=CrnxPfTcm7Y7ZGl7&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=09ydjpBYibdBZDv8MEmG6Upb0Rby8D


Enter code obtained on signing in using SAML:  ············································································································································································································································


/anaconda3/envs/py382/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'austin.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Retrieve locations of signalized intersections intersecting operational maintenance areas to publish into a folder and excel worksheet.

In [ ]:
# This intersect analysis has already been completed
#signals = FeatureLayer(URL.format("TRANSPORTATION_signals2"))
#oma = FeatureLayer(URL.format("Signs_Markings_OMA")) #OMA layer will be on AGOL
#analysis = overlay_layers(oma,signals,overlay_type="Intersect", output_name=overhead)

Retrieve overhead feature layer.

In [ ]:
fl = pd.DataFrame.spatial.from_layer(FeatureLayer(URL.format(overhead))).set_index("SIGNAL_ID")

Query and filter dataset. Add columns for Google URL, N,S,E,W, and Created By.

In [33]:
df = fl.query("FY_OVERHEAD_SIGN_MAINT == {}".format(str(YEAR)).filter(items=col)
g_url = "https://www.google.com/maps/place/{},+Austin,+TX"
gmap = df['LOCATION_NAME'].apply(lambda x: g_url.format(x.lstrip().replace(" / ","+%26+").replace(" ","+").split("(", 1)[0]))
gmap = pd.DataFrame(gmap).rename(columns={"LOCATION_NAME":"GOOGLE_URL"})
df = pd.concat([df.join(gmap),pd.DataFrame(columns=DIRECTIONS)],sort=False)
df['CREATED_BY'] = AUTHOR
df.to_excel(TABLE_NAME, sheet_name='OMA_Signals')
display(df.head()) # Display first 5 rows

,COA_INTERSECTION_ID,FY_OVERHEAD_SIGN_MAINT,LOCATION_NAME,LATITUDE,LONGITUDE,GOOGLE_URL,N,S,E,W,CREATED_BY
88,5157283.0,2020,BURNET RD / HANCOCK DR,None,None,https://www.google.com/maps/place/BURNET+RD+%2...,NaN,NaN,NaN,NaN,Susanne Gov
104,5155653.0,2020,LAMAR BLVD / PARKWAY,None,None,https://www.google.com/maps/place/LAMAR+BLVD+%...,NaN,NaN,NaN,NaN,Susanne Gov
111,5155479.0,2020,35TH ST / JEFFERSON ST,None,None,https://www.google.com/maps/place/35TH+ST+%26+...,NaN,NaN,NaN,NaN,Susanne Gov
205,5153995.0,2020,6TH ST / CAMPBELL ST,None,None,https://www.google.com/maps/place/6TH+ST+%26+C...,NaN,NaN,NaN,NaN,Susanne Gov
209,5154661.0,2020,12TH ST / WEST LYNN ST,None,None,https://www.google.com/maps/place/12TH+ST+%26+...,NaN,NaN,NaN,NaN,Susanne Gov


In [29]:
# Estimate photos to to input manually. Multiplied by 4 by number of signal poles.
df['GOOGLE_URL'].count() * 4

548

## Part 2: Retrieve Photos from URL

After the excel spreadsheet has been created, <b>manually retrieve the google streetview urls of overhead signal poles for 2020 maintenance.</b> The goal of collecthing these URLS is to eventually generate photos of overhead traffic signs in need of maintenance.

In [3]:
df = pd.read_excel(TABLE_NAME,index_col=0)

In [4]:
options = Options()
options.add_argument("--headless")

In [ ]:
driver = webdriver.Chrome(FILE_PATH + r'/chromedriver',options=options)
driver.set_window_size(1600, 800) # set photo sizing
for index, row in df.iterrows():
    for x in DIRECTIONS:
        link = row[x]
        if type(link) == str and link != "N/A":
            driver.get(link)
            driver.execute_script(open(FILE_PATH + r"/hide_ui.js").read())
            time.sleep(4) # wait for page to load
            screenshot = driver.save_screenshot(FILE_PATH + r'/SignPhotos/{}_{}.png'.format(index,x))
driver.quit()

In [4]:
link = "https://goo.gl/maps/5LqR3NoVdF6938cQA"
driver = webdriver.Chrome(FILE_PATH + r'/chromedriver',options=options)
driver.set_window_size(1600, 800) # set photo sizing
driver.get(link)
driver.execute_script(open(FILE_PATH + r"/hide_ui.js").read())
time.sleep(3) # wait for page to load
screenshot = driver.save_screenshot(FILE_PATH + r'/SignPhotos/355_S.png')
driver.quit()

In [21]:
display(df.head())

,COA_INTERSECTION_ID,FY_OVERHEAD_SIGN_MAINT,LOCATION_NAME,LATITUDE,LONGITUDE,GOOGLE_URL,LINK,N,S,E,W,CREATED_BY
88,5157283,2020,BURNET RD / HANCOCK DR,NaN,NaN,https://www.google.com/maps/place/BURNET+RD+%2...,https://www.google.com/maps/place/BURNET+RD+%2...,NaN,NaN,NaN,NaN,Susanne Gov
104,5155653,2020,LAMAR BLVD / PARKWAY,NaN,NaN,https://www.google.com/maps/place/LAMAR+BLVD+%...,https://www.google.com/maps/place/LAMAR+BLVD+%...,https://goo.gl/maps/52cUefr9qhs1Jb2U8,https://goo.gl/maps/kh62J1KTq46C6cpT6,NaN,NaN,Susanne Gov
111,5155479,2020,35TH ST / JEFFERSON ST,NaN,NaN,https://www.google.com/maps/place/35TH+ST+%26+...,https://www.google.com/maps/place/35TH+ST+%26+...,https://goo.gl/maps/DpaB8rqj2Lf2U2av9,https://goo.gl/maps/vJsALCV3zJKce9cBA,https://goo.gl/maps/SQMjudY1dz4NRNnv9,https://goo.gl/maps/YAUA7TnakpgK61Hy7,Susanne Gov
205,5153995,2020,6TH ST / CAMPBELL ST,NaN,NaN,https://www.google.com/maps/place/6TH+ST+%26+C...,https://www.google.com/maps/place/6TH+ST+%26+C...,https://goo.gl/maps/KXQWi8eaqSrZU4ug8,https://goo.gl/maps/Pyx1rpRorPyN512w5,NaN,https://goo.gl/maps/mdM4HZ2VK6WXGqiUA,Susanne Gov
209,5154661,2020,12TH ST / WEST LYNN ST,NaN,NaN,https://www.google.com/maps/place/12TH+ST+%26+...,https://www.google.com/maps/place/12TH+ST+%26+...,NaN,NaN,NaN,NaN,Susanne Gov


## Part 3: Create CSV of input photos in deployed model
Using AutoML to label/train images, we can now use the model to detect some of the most common overhead signs in photos.

In [4]:
# Credentials to access Google Applications
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "/Users/SusanneGov/Documents/My First Project-bcfca3ee21b0.json"

# Used to create predictio from deployed model
def get_prediction(content):
    project_id = "micro-progress-275116"
    model_id = "IOD7288297542666682368"
    prediction_client = automl_v1beta1.PredictionServiceClient()
    name = 'projects/{}/locations/us-central1/models/{}'.format(project_id, model_id)
    payload = {'image': {'image_bytes': content }}
    params = {}
    request = prediction_client.predict(name, payload, params)
    return request  # waits till request is returned

We can use the `get_prediction` method to detect the following overhead signs:
- Street Name Sign
- Left Turn Signal
- Left Turn Yield on Flashing Yellow
- Left Turn Yield on Green
- Left Turn Arrow Only

In [215]:
# Creates empty dictionary to input object detection info
info = {}
# for loop of all photos in folder
for filename in os.listdir(FILE_PATH + "/SignPhotos"):
    if filename.endswith(".png"): 
        with open(FILE_PATH + "/SignPhotos/" + filename, "rb") as content_file:
            content = content_file.read()
        data = get_prediction(content)
        name = set() # to prevent duplicate sign detection from other directions
        for result in data.payload:
            name.add(result.display_name)
        info[filename] = name

After prediction information has been collected (~17 minutes), we will export it onto a spreadsheet.

In [234]:
temp_df = pd.DataFrame.from_dict(info,orient="index").sort_index()
temp_df.to_excel(FILE_PATH + r"/results.xlsx", sheet_name='Sign Objects')
display(temp_df)

,0,1,2
1004_E.png,None,None,None
1004_S.png,None,None,None
1015_E.png,Street_Name_Sign,None,None
1015_N.png,left_turn_yield_on_green,Street_Name_Sign,None
1015_S.png,Street_Name_Sign,None,None
...,...,...,...
955_N.png,Street_Name_Sign,None,None
955_S.png,left_turn_yield_on_green,Street_Name_Sign,None
964_N.png,Street_Name_Sign,None,None
964_S.png,Street_Name_Sign,None,None


The results spreadsheet requires QA/QC work to be done. Once that it complete, the last step is to create the work order document.

In [73]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from subprocess import  Popen

Create dictionaries of intersection information and overhead signs identified.

In [43]:
df = pd.read_excel(FILE_PATH + r"/OverheadSigns_FY2020.xlsx").rename(columns={'Unnamed: 0':'Signal ID'})
cover = {d['Signal ID']: [d['COA_INTERSECTION_ID'],d['LOCATION_NAME']] for d in df.to_dict(orient='records')}
df_signs = pd.read_excel(FILE_PATH + r"/results.xlsx")
table = {d['File name']: [d['Sign 1'],d['Sign 2'],d['Sign 3'],d['Sign 4'],d['Sign 5']] for d in df_signs.to_dict(orient='records')}

Identify 

In [74]:
directions = ["N","S","E","W"]
for index, row in df.iterrows():
    workOrder = Presentation()
    for d in directions:
        slide = workOrder.slides.add_slide(workOrder.slide_layouts[6])
        slide = slide.shapes
        # Create Header
        title = slide.add_textbox(Inches(0.25),Inches(0.4),Inches(7.5),Inches(0.75))
        tf = title.text_frame
        tf.clear()
        p = tf.paragraphs[0]
        run = p.add_run()
        run.text = "Austin Transportation Department Signs Work Orders"
        font = run.font
        font.name = 'Arial'
        font.size = Pt(20)
        font.bold = True
        pic = slide.add_picture(FILE_PATH + r"/seal.png",Inches(8.5),Inches(0.25))

        # Insert Image
        try:
            pic = slide.add_picture(FILE_PATH + r"/SignPhotos/" + str(row['Signal ID']) + "_" + d + ".png",Inches(0.25),Inches(1.1))
            line = pic.line
            line.color.rgb = RGBColor(0,0,0)
            line.width = Inches(0.05)
        except IOError:
            pass 
        
        # Insert general information
        info = slide.add_textbox(Inches(8.25),Inches(1.15),Inches(1.5),Inches(4.5))
        infotext = info.text_frame.paragraphs[0]
        info.text_frame.word_wrap = True
        run1 = infotext.add_run()
        run1.text = "Location Name:\n" + row['LOCATION_NAME'] + "\nIntersection ID:" + str(row['COA_INTERSECTION_ID'])
        run1.font.size = Pt(12)
        run1.font.bold = True
        run1.font.underline = True
        run = infotext.add_run()
        run.text = "Cardinal Direction: "
        run.font.bold = True
        run.font.underline = True  
        run.font.size = Pt(12)
        c = infotext.add_run()
        c.text = d
        c.font.size = Pt(12) 
    workOrder.save(FILE_PATH + r"/WorkOrders2020PPTX/" + str(i) + '.pptx')

After work oder file has been created, convert it into PDF form.

In [75]:
import win32com.client
for filename in os.listdir(FILE_PATH + r"/WorkOrders2020PPTX"):
    if filename.endswith(".pptx"): 
        powerpoint = win32com.client.Dispatch("Powerpoint.Application")
        pdf = powerpoint.Presentations.Open(FILE_PATH + r"/WorkOrders2020PPTX/" + filename,WithWindow=False)
        pdf.SaveCopyAs(FILE_PATH + r"/WorkOrders2020PPTX/" + filename[:-5] + ".pdf" ,32)
        pdf.Close()
        powerpoint.Quit()

ModuleNotFoundError: No module named 'win32com'

In [52]:
# Workflow for 2020
# 1) Merge OMA and signals - waiting for OMA to be published on AGOL/GISMaint
# 2) Filter df to 2020 Overhead maintenance
# 3) Create dataset of all urls
# 4) Input overhead photo urls
# 5) Create for loop for screengrab
# 6) Create table of work orders
#
#
# VVVV              BELOW IS PART OF OLD SCRIPT             VVVV

In [18]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from PIL import Image
from resizeimage import resizeimage
from collections import OrderedDict
import datetime
import sys
import os
import win32com.client

In [5]:
# This will create the list of values for each intersection based on signal ID
filePath = r"G:\ATD\ATD_GIS\Signs\123_Signs_Maintenance_Plan\Operational_Maintenance_Areas_Sign_Maintenance_Plan"
author = "Susanne Gov"

df = pd.read_excel(filePath + r"\OverheadSigns_FY2019.xlsx", sheet_name='Signs')  # sheetname is optional
df.to_csv(filePath + r'\OverheadSigns_FY2019_SIGNS.csv', index=False)

work_orders = pd.read_excel(filePath + r"\OverheadSigns_FY2019.xlsx", sheet_name='StreetView Link')  # sheetname is optional
work_orders.to_csv(filePath + r'\OverheadSigns_FY2019.csv', index=False)          
signalID = []
index = 1
with open (filePath + r'\OverheadSigns_FY2019.csv','r') as csvfile:
    orderRow = csv.reader(csvfile, delimiter=',')
    for row in orderRow:
        values = OrderedDict()
        if row[0] != "SIGNAL_ID":
            values["Created Date"] = row[15]
            values["Created By"] = author
            values["Work Order ID"] = row[14]
            values["Signal ID"] = row[0]
            values["Intersection ID"] = row[1]
            values["Primary Street"] = row[2]
            values["Cross Street"] = row[3]
            values["Link (N)"] = row[10]
            values["Link (S)"] = row[11]
            values["Link (E)"] = row[12]
            values["Link (W)"] = row[13]
            signalID.append(values)
            index +=1
    index = 1
    csvfile.close()

In [3]:
signs = []

with open (filePath + r'\OverheadSigns_FY2019_SIGNS.csv','rU') as csvfile:
    csv = csv.reader(csvfile, delimiter=',')
    for row in csv:
        sign = OrderedDict()
        sign["Signal ID"] = row[0]
        sign["Direction"] = row[1]
        sign["Sign Type"] = row[2] 
        sign["Street Sign"] = row[3]
        sign["Bottom Text (Optional)"] = row[6]
        sign["Install/Remove"] = row[7]
        signs.append(sign)

In [4]:
# Take an existing image of the streetview imagery and will resize the image to fit the work order template
def makeImg (sigId, nSlide):
    imgname = sigId + "_" + direction
    imagery = imagePath + "/" + imgname + ".png"
    try:
        with open(imagery, 'r+b') as f:
            with Image.open(f) as image:
                cover = resizeimage.resize_contain(image, [560, 280])
                cover.save(imagePath + "Cropped/" + imgname + ".png", image.format)
        pic = nSlide.add_picture(imagePath + "Cropped/" + imgname + ".png",Inches(0.25),Inches(1.1))
        line = pic.line
        line.color.rgb = RGBColor(0,0,0)
        line.width = Inches(0.05)
    except IOError:
        return None        

In [5]:
# Create a pretty header
def makeHead(nSlide):
    title = nSlide.add_textbox(Inches(0.25),Inches(0.4),Inches(7.5),Inches(0.75))
    tf = title.text_frame
    tf.clear()
    p = tf.paragraphs[0]
    run = p.add_run()
    run.text = "Austin Transportation Department Signs Work Orders"
    font = run.font
    font.name = 'Arial'
    font.size = Pt(20)
    font.bold = True
    seal = "G:/ATD/ATD_GIS/03_RESOURCES/Seals_Logos/cityseal_60_x_60.jpg"
    pic = nSlide.add_picture(seal,Inches(8.5),Inches(0.25))

In [6]:
# Create information box for work orders
def makeInfo(sigID, nSlide, direction):
    info = nSlide.add_textbox(Inches(8.25),Inches(1.15),Inches(1.5),Inches(4.5))
    infotext = info.text_frame.paragraphs[0]
    info.text_frame.word_wrap = True
    for (key,val) in sigID.iteritems():
        if "Link" not in key or direction in key:
            run1 = infotext.add_run()
            run1.text = key + "\n"
            run1.font.size = Pt(12)
            run1.font.bold = True
            run1.font.underline = True
            run2 = infotext.add_run();
            if "Link" in key and "N/A" not in val:
                run2.hyperlink.address = val
            run2.text =  val + "\n"
            run2.font.size = Pt(12)
    run3 = infotext.add_run()
    run3.text = "Cardinal Direction: "
    run3.font.bold = True
    run3.font.underline = True  
    run3.font.size = Pt(12)
    c = infotext.add_run()
    if direction is "N":
        c.text = "North"
    elif (direction is "S"):
        c.text = "South"
    elif (direction is "E"):
        c.text = "East"
    elif (direction is "W"):
        c.text = "West"
    c.font.size = Pt(12)   

In [7]:
# For changing table font sizes
def iter_cells(table):
    for row in table.rows:
        for cell in row.cells:
            yield cell
# Create list of Sign Overheads
def makeSignTable(sigID,nSlide,direction,signs):
    # row/column/left/top/width/height
    table = nSlide.add_table(6,4,Inches(0.25),Inches(5.2),170,1000).table
    table.cell(0,0).text = "Sign Type"
    table.cell(0,1).text = "Street Sign"
    table.cell(0,2).text = "Bottom Text"
    table.cell(0,3).text = "Install/Remove"
    
    table.columns[0].width = Inches(5)
    table.columns[1].width = Inches(1.75)
    table.columns[2].width = Inches(1.5)
    table.columns[3].width = Inches(1.3)
    
    index = 1
    for x in signs:
        if x["Signal ID"] == sigID and x["Direction"] == direction:
            table.cell(index,0).text = x["Sign Type"]
            table.cell(index,1).text = x["Street Sign"]
            table.cell(index,2).text = x["Bottom Text (Optional)"]
            table.cell(index,3).text = x["Install/Remove"]
            index +=1
    for cell in iter_cells(table):
        for paragraph in cell.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(12)

In [8]:
# Converts Powerpoint file to PDF file
def convertPDF(path,name):
    in_file = path + "/" + name
    out_file = path + "PDF\\" + name[:-5] + ".pdf"
    powerpoint = win32com.client.Dispatch("Powerpoint.Application")
    pdf = powerpoint.Presentations.Open(in_file,WithWindow=False)
    pdf.SaveCopyAs(out_file,32)
    pdf.Close()
    powerpoint.Quit()

In [9]:
dirs = ["N","S","E","W"]
path = "G:\\ATD\\ATD_GIS\\Signs\\123_Signs_Maintenance_Plan\\Operational_Maintenance_Areas_Sign_Maintenance_Plan\\WorkOrderSignsOverheadFY2019"
imagePath = "G:/ATD/ATD_GIS/Signs/123_Signs_Maintenance_Plan/Operational_Maintenance_Areas_Sign_Maintenance_Plan/SignsFY2019Imagery"

for valueDictionary in signalID:
    workOrder = Presentation()
    dirindex = 0
    for direction in dirs:
        if "N/A" not in valueDictionary["Link ("+ direction + ")"]:
            blank_slide_layout = workOrder.slide_layouts[6]
            slide = workOrder.slides.add_slide(blank_slide_layout)
            nSlide = slide.shapes
            makeImg(valueDictionary["Signal ID"], nSlide)
            makeSignTable(valueDictionary["Signal ID"],nSlide,direction,signs)
            makeHead(nSlide)
            makeInfo(valueDictionary, nSlide, direction)
        dirindex += 1
    name = "ATDSignsWorkOrders_" + valueDictionary["Signal ID"] + ".pptx"
    workOrder.save(str(path) + "/" + name)
    convertPDF(path,name)